## X-track plotting

### Imports

In [ ]:
# Import standard libraries
import numpy as np
from cpymad.madx import Madx
import json
import xtrack as xt
import xfields as xf
import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio
import plotly.express as px

# Import plotting functions
from plotting_functions import return_plot_lattice_with_tracking

# Nicer plots
pio.templates.default = "plotly_white"

# Different backend
import plotly.io as pio
# pio.renderers.default = "vscode"

# Color list [blue, red, green, purple, orange,light blue, pink, light green, light violet, yellow]
l_colors = px.colors.qualitative.Plotly




### Load json files, build trackers and corresponding dataframes

In [ ]:
# Load the lines into json files
with open("json_lines/line_b1.json", "r") as fid:
    dct_b1 = json.load(fid)
    line_b1 = xt.Line.from_dict(dct_b1)

with open("json_lines/line_b4.json", "r") as fid:
    dct_b4 = json.load(fid)
    line_b4 = xt.Line.from_dict(dct_b4)

# Build trackers
tracker_b1 = line_b1.build_tracker()
tracker_b4 = line_b4.build_tracker()

# Build a dataframe with the elements of the lines
df_elements_b1 = pd.DataFrame([x.to_dict() for x in line_b1.elements])
df_elements_b4 = pd.DataFrame([x.to_dict() for x in line_b4.elements])

df_elements_b1


### Get surveys and beam Twiss parameters


In [ ]:
# Get survey dataframes
df_sv_b1 = tracker_b1.survey().to_pandas()
df_sv_b4 = tracker_b4.survey().to_pandas()

# Get Twiss dataframes
df_tw_b1 = tracker_b1.twiss().to_pandas()
df_tw_b4 = tracker_b4.twiss().to_pandas()

# Reverse x-axis only for beam 1
df_sv_b1["X"] = -df_sv_b1["X"]
df_tw_b1["x"] = -df_tw_b1["x"]


### Add missing data to the dataframes of elements due to thin lens approximation

In [ ]:
def return_dataframe_corrected_for_thin_lens_approx(df_elements, df_tw):
    df_elements_corrected = df_elements.copy(deep=True)

    # Add all thin lenses (length + strength)
    for i, row in df_tw.iterrows():
        if ".." in row["name"]:
            name = row["name"].split("..")[0]
            index = df_tw[df_tw.name == name].index[0]

            # Add length
            if np.isnan(df_elements_corrected.loc[index]["length"]):
                df_elements_corrected.at[index, "length"] = 0.0
            df_elements_corrected.at[index, "length"] += df_elements.loc[i]["length"]

            # Add strength
            if np.isnan(df_elements_corrected.loc[index]["knl"]).all():
                df_elements_corrected.at[index, "knl"] = (
                    np.array([0.0] * df_elements.loc[i]["knl"].shape[0], dtype=np.float64)
                    if type(df_elements.loc[i]["knl"]) != float
                    else 0.0
                )
            df_elements_corrected.at[index, "knl"] = (
                df_elements_corrected.loc[index, "knl"] + np.array(df_elements.loc[i]["knl"])
                if type(df_elements.loc[i]["knl"]) != float
                else df_elements.loc[i]["knl"]
            )

            # Replace order
            df_elements_corrected.at[index, "order"] = df_elements.loc[i]["order"]

            # Drop row
            df_elements_corrected.drop(i, inplace=True)

    return df_elements_corrected


df_elements_b1_corrected = return_dataframe_corrected_for_thin_lens_approx(df_elements_b1, df_tw_b1)
df_elements_b4_corrected = return_dataframe_corrected_for_thin_lens_approx(df_elements_b4, df_tw_b4)
df_elements_b1_corrected


### Get indices between IP8 and IP2, and IP4 and IP6


In [ ]:
idx_ip4_b1 = df_tw_b1.loc[df_tw_b1['name'] == 'ip4'].index[0]
idx_ip6_b1 = df_tw_b1.loc[df_tw_b1['name'] == 'ip6'].index[0]
idx_ip8_b1 = df_tw_b1.loc[df_tw_b1['name'] == 'ip8'].index[0]
idx_ip2_b1 = df_tw_b1.loc[df_tw_b1['name'] == 'ip2'].index[0]
l_indices_to_keep = list(range(idx_ip4_b1,idx_ip6_b1)) + list(range(idx_ip8_b1,idx_ip2_b1))

### Make interactive plot for the lattice

In [ ]:
fig = return_plot_lattice_with_tracking(
    df_sv_b1,
    df_elements_b1_corrected,
    df_tw_b1,
    df_sv_4 = df_sv_b4,
    df_tw_4 = df_tw_b4,
    l_indices_to_keep=l_indices_to_keep)
    
config = {'displayModeBar': True, 'displaylogo': False}
fig.show(config=config)
#fig.write_html("layout.html")



### Plot phase-space dynamics in 2D

In [ ]:
# Build figure
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=df_tw_b1['mux'][:10000],
        y=(df_tw_b1['x']/df_tw_b1['betx']**0.5)[:10000] ,
        mode="lines",
        showlegend=True,
        name = r'$\frac{x}{\sqrt{\beta_x}}$',
    )
)

fig.add_trace(
    go.Scatter(
        x=df_tw_b1['muy'][:10000],
        y=(df_tw_b1['y']/df_tw_b1['bety']**0.5)[:10000],
        mode="lines",
        showlegend=True,
        name = r'$\frac{y}{\sqrt{\beta_y}}$',
    )
)
cnt = 0
for i, row in df_ip.iterrows():
    theta = np.pi + row["theta"]
    fig.add_trace(
        go.Scatter(
            mode="markers",
            x=[df_tw_b1['muy'][i] ],
            y=[0 ],
            marker_symbol=218,
            marker_line_color="midnightblue",
            marker_color="grey",
            marker_line_width=2,
            marker_size=15,
            name=row["name"],
            showlegend=False,
            legendgroup="IP",
        )
    )
    cnt+=1
    if cnt>3:
        break

    

fig.update_layout(
    title_text="Transverse dynamics",
    title_x=0.5,
    showlegend=True,
    #xaxis_range=[df_sv_b1["X"].min() - 1000, df_sv_b1["X"].max() + 1000],
    #yaxis_range=[df_sv_b1["Z"].min() - 1000, df_sv_b1["Z"].max() + 1000],
    xaxis_showgrid=True,
    yaxis_showgrid=True,
    xaxis_title=r'$\mu_{x,y}$',
    #yaxis_title=r'$p_x$',
    #yaxis_scaleanchor="x",
    #yaxis_scaleratio=1,
    width=1000,
    height=400,
    #margin=dict(l=50, r=50, b=100, t=100, pad=4),
)

# Add range slider
fig.update_layout(
    xaxis=dict(
        rangeslider=dict(
            visible=True
        ),
        type="linear",
        range = (10,20),
    )
)

config = {'displayModeBar': False}
fig.show(config=config)



In [8]:
import plotly.graph_objects as go

import networkx as nx

G = nx.random_geometric_graph(20, 0.5)



edge_x = []
edge_y = []
for edge in G.edges():
    x0, y0 = G.nodes[edge[0]]['pos']
    x1, y1 = G.nodes[edge[1]]['pos']
    edge_x.append(x0)
    edge_x.append(x1)
    edge_x.append(None)
    edge_y.append(y0)
    edge_y.append(y1)
    edge_y.append(None)

edge_trace = go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=0.5, color='#888'),
    hoverinfo='none',
    mode='lines')

node_x = []
node_y = []
for node in G.nodes():
    x, y = G.nodes[node]['pos']
    node_x.append(x)
    node_y.append(y)

node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers',
    hoverinfo='text',
    marker=dict(
        showscale=True,
        # colorscale options
        #'Greys' | 'YlGnBu' | 'Greens' | 'YlOrRd' | 'Bluered' | 'RdBu' |
        #'Reds' | 'Blues' | 'Picnic' | 'Rainbow' | 'Portland' | 'Jet' |
        #'Hot' | 'Blackbody' | 'Earth' | 'Electric' | 'Viridis' |
        colorscale='YlGnBu',
        reversescale=True,
        color=[],
        size=10,
        colorbar=dict(
            thickness=15,
            title='Node Connections',
            xanchor='left',
            titleside='right'
        ),
        line_width=2))


node_adjacencies = []
node_text = []
for node, adjacencies in enumerate(G.adjacency()):
    node_adjacencies.append(len(adjacencies[1]))
    node_text.append('# of connections: '+str(len(adjacencies[1])))

node_trace.marker.color = node_adjacencies
node_trace.text = node_text

In [11]:
fig = go.Figure(data=[edge_trace, node_trace],
             layout=go.Layout(
                title='<br>Network graph made with Python',
                titlefont_size=16,
                showlegend=False,
                hovermode='closest',
                margin=dict(b=20,l=5,r=5,t=40),
                annotations=[ dict(
                    text="Python code: <a href='https://plotly.com/ipython-notebooks/network-graphs/'> https://plotly.com/ipython-notebooks/network-graphs/</a>",
                    showarrow=True,
                    xref="paper", yref="paper",
                    x=0.005, y=-0.002 ) ],
                xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
                )
fig.show()